In [3]:
import asyncio
import json
from datetime import datetime
from websockets import connect
from termcolor import cprint
import random

In [5]:
symbols=['btcusdt','ethusdt','solusdt','bnbusdt','dogeusdt','wifusdt']
websocket_url_base='wss://fstream.binance.com/ws/'

In [ ]:
import asyncio
import json
from datetime import datetime

from termcolor import cprint
from websockets import connect  # pip install websockets

# Make sure these exist somewhere above:
symbols = ["btcusdt", "ethusdt", "solusdt", "bnbusdt"]
websocket_url_base = "wss://fstream.binance.com/ws/"

shared_symbol_counter = {"count": 0}
print_lock = asyncio.Lock()


async def binance_funding_stream(symbol: str, shared_counter: dict):
    """
    Stream markPrice updates for a symbol and print funding info.
    """
    global print_lock

    websocket_url = f"{websocket_url_base}{symbol}@markPrice"

    async with connect(websocket_url) as websocket:
        while True:
            try:
                message = await websocket.recv()
                data = json.loads(message)

                # time
                event_time = datetime.fromtimestamp(
                    data["E"] / 1000
                ).strftime("%H:%M:%S")

                # symbol + funding
                symbol_display = data["s"].replace("USDT", "")
                funding_rate = float(data["r"])            # per 8h funding
                yearly_funding_rate = (funding_rate * 3 * 365) * 100  # % per year

                # colour logic
                if yearly_funding_rate > 50:
                    text_color, back_color = "black", "on_red"
                elif yearly_funding_rate > 30:
                    text_color, back_color = "black", "on_yellow"
                elif yearly_funding_rate > 5:
                    text_color, back_color = "black", "on_cyan"
                elif yearly_funding_rate < -10:
                    text_color, back_color = "black", "on_green"
                else:
                    text_color, back_color = "black", "on_white"

                # only one task prints at a time
                async with print_lock:
                    cprint(
                        f"{symbol_display} funding: {yearly_funding_rate:.2f}%",
                        text_color,
                        back_color,
                    )

                # shared counter across all streams
                shared_counter["count"] += 1

                # once we've printed at least once per symbol, print a marker
                if shared_counter["count"] >= len(symbols):
                    async with print_lock:
                        cprint(f"{event_time} yrly fund", "white", "on_black")
                    shared_counter["count"] = 0

            except Exception as e:
                async with print_lock:
                    cprint(f"Error in funding stream {symbol}: {e}", "white", "on_red")
                await asyncio.sleep(5)


async def main():
    tasks = [
        binance_funding_stream(symbol, shared_symbol_counter)
        for symbol in symbols
    ]
    await asyncio.gather(*tasks)


if __name__ == "__main__":
    asyncio.run(main())
